In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import itertools

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import bottleneck as bn
import pandas as pd
# from sklearn import decomposition
import networkx as nx
# import networkx.algorithms.approximation as nxa
import igraph as ig
# # http://stackoverflow.com/questions/35279733/what-could-cause-networkx-pygraphviz-to-work-fine-alone-but-not-together
# from networkx.drawing.nx_agraph import graphviz_layout
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid", {"axes.facecolor": ".95"})
import missingno as msno
%matplotlib inline

from collections import defaultdict, Counter
from IPython.display import display as disp

from speclib.loaders import (loadUserPhonenumberDict, getUserList, Useralias,
                             loadUserParallel, dict2DataFrame, users2DataFrame)
from speclib.plotting import looseAxesLimits, barSBS, countsOnBarPlot, plotNeatoGraph, nxQuickDraw
from speclib.graph import networkx2igraph, igraph2networkx, userDF2nxGraph, userDF2activityDataframe
from speclib.misc import nanEqual, timedelta2unit, standardizeData

for k, v in {'font.size': 13.0,
             'legend.fontsize': 13.0,
             'axes.labelsize': 12.0,
             'axes.titlesize': 15.0,
             'figure.figsize': [16.0, 7.0],
             'figure.dpi': 200,
             'figure.titlesize': 'large',
             'xtick.labelsize': 13.0,
             'ytick.labelsize': 13.0}.items():
    mpl.rcParams[k] = v

%load_ext watermark
%watermark -a "Allan Leander Rostock Hansen" -u -d -v -p numpy,bottleneck,pandas,matplotlib,sklearn,missingno
%watermark  -p networkx,igraph,seaborn

# Load and clean data

In [ ]:
if os.path.isfile('phone_df.h5'):
    df = pd.io.pytables.read_hdf('phone_df.h5', 'df')
    import pickle
    with open('useralias.pk', 'br') as fid:
        ua = pickle.load(fid)
else:
    ua = Useralias()
    userSpec = [(user, ua[user], ('sms', 'call')) for user in getUserList()]
    userData = loadUserParallel(userSpec) 
    df = users2DataFrame(userData, ua)
    del userData
phonebook = loadUserPhonenumberDict(ua) 
df.head() 

Remove call to users not in phonebook.

In [ ]:
df = df[df.number.isin(phonebook)] 

Add _contactedUser_ column and remove the _number_ column.

In [ ]:
df['contactedUser'] = df.number.apply(lambda x: phonebook[x]) 
df = df.drop('number', axis=1)

In [ ]:
df.head() 

## Check for obvious outliers

In [ ]:
ax = df.timestamp.dt.year.value_counts().sort_index(ascending=False).plot.bar() 
countsOnBarPlot(ax) 

Remove data preceding 2013.

In [ ]:
df = df[df.timestamp.dt.year >= 2013] 

## Remove users contacting themself

In [ ]:
tmp = df.reset_index()
tmp = tmp[(tmp.user != tmp.contactedUser)]
df = tmp.set_index(['user', 'comtype'])

# Turn data into a Networkx graph

In [ ]:
aDct = dict()
for i, user in enumerate(df.index.get_level_values('user').unique()):
    aDct[user] = df.loc[user].contactedUser.value_counts().to_dict() 

In [ ]:
adf = pd.DataFrame.from_dict(aDct, orient='index')
adf.sort_index(axis=1, inplace=True)
adf.head()

In [ ]:
g = nx.Graph(aDct)

Ensure that the graph contains the correct number of nodes

In [ ]:
assert len(list(g.nodes())) == len(set(df.index.get_level_values('user').tolist() + df.contactedUser.tolist())) 

## Analyse the graph

In [ ]:
fig, ax = plt.subplots()
cnt = Counter(el[1] for el in g.degree())
x, y = list(zip(*((i, cnt[i]) for i in range(max(cnt)+1))))
ax.bar(x, y)
ax.set_xlabel('Node degree')
ax.set_ylabel('Counts')
countsOnBarPlot(ax) 

# Clique detection

In [ ]:
cliqueDf = pd.DataFrame(nx.clique.find_cliques_recursive(g))

cliqueDf['cliqueSize'] = cliqueDf.count(axis=1)
cliqueDf = cliqueDf.sort_values('cliqueSize', ascending=False)

ax = cliqueDf.cliqueSize.value_counts().sort_index().plot.bar(rot=0) 
ax.set_xlabel('Clique size')
ax.set_ylabel('Counts') 

## Choose a clique with 5 users and make a subgraph 

In [ ]:
chosenUserLst = cliqueDf[cliqueDf.cliqueSize == 5].drop('cliqueSize', axis=1).iloc[0].dropna().tolist()
print("Chosen users:", *chosenUserLst, sep='\n')
gs = g.subgraph(chosenUserLst)

In [ ]:
nxQuickDraw(gs)

In [ ]:
chosenUserLst